### Tema "Kvinde og dannelse" med ordsekker
Kode utviklet av Lars G.Johnsen

#### Boot cell

Skip if `module_update.py` is in same folder as notebook, next cell downloads this file which in turn will download other files.

In [1]:
!curl "https://raw.githubusercontent.com/Yoonsen/Modules/master/module_update.py" > "module_update.py"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3823  100  3823    0     0  14109      0 --:--:-- --:--:-- --:--:-- 15110


#### Get essentials

Import text and graph modules for analysis, and css for display

In [2]:
from module_update import update, css, printmd

In [3]:
css()

# Generisk analyse

Intro

### Start med å importere modulene

In [5]:
update('nbtext', overwrite=True)
update('graph_networkx_louvain')
import nbtext as nb
import graph_networkx_louvain as gnl
from graph_networkx_louvain import cutdown, make_collocation_graph

Updated file `/Users/heidikarlsen/Documents/Doktorgrad/Jupyter Notebook/nbtext.py`

Updated file `/Users/heidikarlsen/Documents/Doktorgrad/Jupyter Notebook/graph_networkx_louvain.py`

ModuleNotFoundError: No module named 'community'

In [6]:
def link(urn):
    return "https://www.nb.no/items/URN:NBN:no-nb_digibok_{u}".format(u = urn)

Her er en uvektet jaccard som sjekker de radene som har mer enn ett visst antall treff, bestemt av parameteret `limit`.

In [7]:
def interesting_urns(df, limit):
    jaccard = dict((df > 0).astype(int).sum(axis=1))
    res = []
    for urn in jaccard:
        if jaccard[urn] >= limit:
            res.append(urn)
    return res

In [8]:
def get_para(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs

In [9]:
def interesting_paragraphs(urns, wordbag, limit):
    interest = dict()
    for u in urns:
        paragraphs = nb.wordbag_eval_para(wordbag, [u])
        for x in paragraphs:
            if len(paragraphs[x]) >= limit:
                interest[x] = paragraphs[x]
    return interest

In [10]:
import requests
def get_paragraphs(urn, paras):
    """Return paragraphs for urn"""
    param = dict()
    param['paragraphs'] = paras
    param['urn'] = urn
    r = requests.get("https://api.nb.no/ngram/paragraphs", json=param)
    return r.json()

## Bygg korpus og analysér

Hent URN-er fra tekst eller metadata

### Hent ut bøker fra 1830 - 1880

In [11]:
urns =nb.get_urn({'year': 1830, 'next':50, 'limit':10000}) 

In [12]:
len(urns)

6803

## Ordsekker

Definer hyperleksemer som ordsekker. Definisjonen er på formen 
"""definisjon : ord1, ord2, ..., ordN;"""
Kommandoen `def2dict` konverterer definisjonen til en pythonstruktur som kan brukes videre. Den vil samtidig legge til en kapitalisert variant av alle ord som ikke har en.

In [21]:
Kvinde_dannelse = nb.def2dict(
"""Kvinde_: kvinne, kvinnen, kvinner, kvinnene, kvinnerne, kvinnes, kvinnens, kvinners, kvinnenes, kvinnernes, kvinde, kvinden, kvinder, kvinderne, kvindens, kvindes, kvindernes, kvinders, kvindelige, kvindelig, kvindeligt, kvindene, kvindeme, kvindekøn, kvindenes, kvindemes, kvindc, kvindcr, kvindcn, kvindcns, kvindcs, kvindcrnes, kvindcrnc, kvindcrs, kvindcrncs, kvindclighed, kvindcligheden, kuindc, kuindcr, kuindcn, kuindcs, kuindcrne, kuindcns, kuinde, kuinder, kuinden, kuinderne, kuindens, kuinders, kuindes, kvindelighed, kvindeligt, kvindelighedens, kvindeligheden, kvindeligheds, kvindelighed, kvindclige, kvindelighcd, kvindeligked, kvindeligbed, qvinde, qvinden, qvinder, qvinderne, qvindene, qvindelig, qvindelige, qvindelighed, quinde, quinden, quinder, qvinderne, qvindene, quindelig, quindelighed, qvindcn, quindcn, qvindclig, quindclig, qvindcr, quindcr, qvindcrne, qvindene, quindcrne, quindcne, qvindclighcd, quindclighcd, qvindelighcd, quindelighcd, qvindclighed, quindclighcd, Dame, Damen, dame, Damen, damc, Damc, damcn, Damcn, damer, Damer, damcr, Damcr, damene, Damene, damcne, Damcne, damcrne, Damcrne, damerne, Damerne, damcrnc, Damcrnc, damcnc, Damcnc, damernc, Damernc, damenc, Damenc, jente, jenten, jenta, jenter, jentene, jenterne, jentas, jentens, jentes, jenters, jentenes, jenterne, jenternes, jcnte, jcnten, jcnterne, pike, piken, piker, pikene, pikerne, pikes, pikens, pikers, pikenes, pikernes, pikebarn, pikebørn, pige, piger, pigene, pigerne, piges, pigens, pigers, pigenes, pigernes, pigebarn, pigebarnet, pigeborn, pigc, pigcn, pigcr, pigcne, pigcrne, pigcs, pigcbarn, pigcbarnet, pigcborn, heltinne, heltinnen, heltinner, heltinnene, heltinnens, hcltinnen, heltinde, heltinden, heltinder, heltindene, heltinderne, heltindens, heltindc, tjenerinne, tjenerinnen, tjenerinde, tjenerinden, tjcnerinde, tjcncrinde, tjcnerinden, tjcncrinden, tjcncrindcn, fristerinne, fristerinnen, fristerinnens, fristerinde, fristerinden, fristerindens, fristcrinde, fristcrinden, fristcrindcn, kone, konen, koncn, hustru, huustru, fruentimmer, fruentimmeret, fruentimmere, fruentimmerne, fruentimmernc, fruentimmers, fruentimmerets, fruentimmeraktig, fruentimmeragtig, fruentimmerlig, fruentimmerlige, fruentimmerfølelser, fruentimre, fruentimret, fruentimrene, qvindekjønnet, qvindekjonnet, qvindckjonnet, quindckjonnet, quindekjønnet, quindekjonnet, quindckjonnet, qvindekjonn, qvindckjonn, quindekjonn, quindckjonn, beskjørtede, bcskjørtede, bcskjørtcde, bcskjortcde, beskjortcde, bcskjortede, beskjortede, fruentimmernetheder, fruentimmcrncdhcer, frucntimmcrncdhcdcr, husmødrene, husmodrene, husmoder, husmor, husmodrcne, mor, moder, moderlig, modre, modrcne, modrc, modcrlig, modcrlighcden, modcrlighed, moderlighed, modcrlighcd, telegrafistinde, telegrafistinder, telegrafistinderne, telcgrafistinde, tclcgrafistinde, tclcgrafistindcr, telcgrafistindcrne, lcererinde, lærerinde, lcercrinde, lcercrinder, lcererinderne, lcercrindene, mor, moder, moderlig, modre, modrcne, modrc, modcrlig, modcrlighcden, modcrlighed, moderlighed, modcrlighcd, telegrafistinde, telegrafistinder, telegrafistinderne, telcgrafistinde, tclcgrafistinde, tclcgrafistindcr, telcgrafistindcrne, lcererinde, lærerinde, lcercrinde, lcercrinder, lcererinderne, lcercrindene, Huusholderske, Huusholdersken, Huusmoder, husmoderlig, husmoderlige, husmoderlighed, husmoderligheden, Husmor, Husmoder, Husmodcr, Husmodcrcn, Husmoderen, Husmodcren, Husmodcren, Huusmodcr, Huusmodre, Huusmoders, Huusmodcr, Huusmodrene, Huusmodcren, Huusmodercn, Huusmodcrs, Husmodre, Husmodrc, Husmoderhjerne, Husmoderkall, Husmoderem, Husmoderms, Huusmodcrcn, Huusholderffe, Huusholderster, Huusholderste, Husfrue, Huusfrue, Huusfrne, Favoritinde, Husslavinde, Emancipationsistinde, Emancipationistindern, Læserinder, Læserinde, Lceserinde, Lceserinder, Læserinden, Leserinder, Leserinde, Læserinders, Lcrserinde; 
Dannelse_: Dannelse, Dannelsens, Dannet, Uddannelse, Uddannelsen, Uddanning, Uddannclse, Uddannclsen, Uddannclscn, Dannclse, Dannclsen, Danclsens, Dannct, Dannclsc, Dannclscn, Danncr, Danncde, Danncs, Danncdc, Danncnde, Utvikling, Utviklingen, Utviklingens, Utviklings, Utvikle, Utvikler, Utvikles, Utviklict, Utviklcr, Utviklcs, Utciklcde, Oplæring, Oplæringen, Oplæringens, Oplærings, Mål, Målet, Målct, Maal, Maalet, Maalct, Formål, Formålet, Formålcne, Formaal, Formaalet, Formaalets, Formaals, Fomaalenes, Kunskab, kunskaben, kunskaber, kunskabsrig, Kundskab, kundskaben, kundskaber, Kundskabsrig, Kunnskab, kunnskaben, kunnskaber, Kunnskabsrig, Kunnskabsrik, Kundskabsrik, Kunskabsrik,  kunskabs, kunnskabens, kundskabere, kundskabssyn, kundskabelig, kunskabere, kundskabss, kunclskaber, Visdom, Visdommen, Visdommens, Visdommcn, Visdommcns, Lcerdom, Lcrrdom, Lærdom, Lerdom, Loerdom, Lardom, Laerdom, Lccrdom, Lwrdom, Leerdom, Lsrdom, Lerrdom, Lserdom, Llrrdom, Lnrdom, Lierdom, Locrdom, Ltrrdom, Lssrdom, LErdom, LcerdoM, LcrrdoM, Llerdom, Lcvrdom, Lecrdom, Aandsdannelse, Aandsdannelsen, Aandsdannelsens, Aandsdannclse, Aandsdannelses, Aandsdannelsc, Aandsdannelst, Aandsdannclsc, AandSdannelse; 
Sjel_: Sjel, Sjelen, Sjels, Sjelens, Sjelelig, Sjelelige, Sjcl, Sjclen, Sjcls, Sjclens, Sjclcn, Sjclcns, Sjclelig, Sjclelige, Kvinnesjel, Kvinnesjelen, Kvinnesjelens, Kvindesjel, Kvindesjelen, Kvindesjelens, Kvindesjæl, Kvindesjælen, Kvindesjælens, Kvindesjæls, Aand, aandelig, Aandsliv, aandsrik, aandfuld, aandclig, aandeli, aandélig Aandelighed, Aandeligt, Aandclige, Aandeliges, Aandeligheds, Aandeligc, Aandclig, Aandcligc, Aandclighed, Aandtlige, Aandcligt, Aandklige, Aandelighcd, vis, vise, visc, intelligens, intelligence, inteligens, inteligent, intelligent, begaved, begavelse, aandsevner, aandsevnerne, Bevidsthed, Tankeliv;"""
)
print(Kvinde_dannelse)

{'Kvinde_': ['kvinne', 'kvinnen', 'kvinner', 'kvinnene', 'kvinnerne', 'kvinnes', 'kvinnens', 'kvinners', 'kvinnenes', 'kvinnernes', 'kvinde', 'kvinden', 'kvinder', 'kvinderne', 'kvindens', 'kvindes', 'kvindernes', 'kvinders', 'kvindelige', 'kvindelig', 'kvindeligt', 'kvindene', 'kvindeme', 'kvindekøn', 'kvindenes', 'kvindemes', 'kvindc', 'kvindcr', 'kvindcn', 'kvindcns', 'kvindcs', 'kvindcrnes', 'kvindcrnc', 'kvindcrs', 'kvindcrncs', 'kvindclighed', 'kvindcligheden', 'kuindc', 'kuindcr', 'kuindcn', 'kuindcs', 'kuindcrne', 'kuindcns', 'kuinde', 'kuinder', 'kuinden', 'kuinderne', 'kuindens', 'kuinders', 'kuindes', 'kvindelighed', 'kvindeligt', 'kvindelighedens', 'kvindeligheden', 'kvindeligheds', 'kvindelighed', 'kvindclige', 'kvindelighcd', 'kvindeligked', 'kvindeligbed', 'qvinde', 'qvinden', 'qvinder', 'qvinderne', 'qvindene', 'qvindelig', 'qvindelige', 'qvindelighed', 'quinde', 'quinden', 'quinder', 'qvinderne', 'qvindene', 'quindelig', 'quindelighed', 'qvindcn', 'quindcn', 'qvindclig

Kommandoen `nb.wordbag()` tar en orddefinisjon og en liste URN-er og søker i dem for en første sjekk. Det søket fungerer som en grov hash-index for avsnittene som sjekkes under.

In [22]:
df = nb.wordbag_eval(Kvinde_dannelse, urns[:1000])

In [24]:
df2 = nb.wordbag_eval(Kvinde_dannelse, [x[0] for x in urns[1000:2000]])

In [25]:
totdf = df.append(df2).sort_values(by='snitt', ascending=False)
totdf.head(20)

,Dannelse_,Kvinde_,Sjel_,snitt
2010021003016,14.0,1916.0,653.0,861.000000
2016022548049,129.0,1175.0,550.0,618.000000
2010011903029,244.0,890.0,716.0,616.666667
2010012603007,50.0,263.0,1451.0,588.000000
2008093012001,125.0,440.0,1193.0,586.000000
2014062048065,195.0,1099.0,452.0,582.000000
2009082403015,44.0,240.0,1389.0,557.666667
2016101129001,461.0,113.0,949.0,507.666667
2012102924020,108.0,1160.0,243.0,503.666667
2016100729002,388.0,126.0,951.0,488.333333


## Jaccard


Interessante URNer er de som treffer med temaordene. Juster antall etter behov.

In [26]:
urns_of_interest = interesting_urns(df[list(Kvinde_dannelse.keys())], 3)
urns_of_interest

['2016022548049',
 '2010012603007',
 '2008093012001',
 '2014062048065',
 '2009082403015',
 '2016101129001',
 '2012102924020',
 '2012102924022',
 '2015010648112',
 '2009092303053',
 '2011060704053',
 '2009092303042',
 '2016102848582',
 '2006081400005',
 '2006112101017',
 '2010020503005',
 '2008111103033',
 '2010092803030',
 '2016031548121',
 '2014090108027',
 '2008111903022',
 '2008110603012',
 '2008102303002',
 '2008100703001',
 '2012080908032',
 '2011051004069',
 '2009081903009',
 '2011061608052',
 '2009032012001',
 '2009092403010',
 '2008090903039',
 '2011090920001',
 '2010021603016',
 '2007082302001',
 '2012052204036',
 '2008122310001',
 '2009020600097',
 '2009072703019',
 '2009031312001',
 '2008110503006',
 '2008111103004',
 '2006083100008',
 '2015061048105',
 '2013080608090',
 '2016042848154',
 '2009080603013',
 '2014060208012',
 '2010052806032',
 '2008092603012',
 '2011012406002',
 '2016060348148',
 '2009031612001',
 '2015072829002',
 '2009092303064',
 '2010052806031',
 '20120411

Slik ser datarammen ut for de mest interessante

In [27]:
df.loc[urns_of_interest]

,Dannelse_,Kvinde_,Sjel_,snitt
2016022548049,129.0,1175.0,550.0,618.000000
2010012603007,50.0,263.0,1451.0,588.000000
2008093012001,125.0,440.0,1193.0,586.000000
2014062048065,195.0,1099.0,452.0,582.000000
2009082403015,44.0,240.0,1389.0,557.666667
2016101129001,461.0,113.0,949.0,507.666667
2012102924020,108.0,1160.0,243.0,503.666667
2012102924022,111.0,1060.0,273.0,481.333333
2015010648112,401.0,437.0,417.0,418.333333
2009092303053,136.0,152.0,929.0,405.666667


Det kutter betydelig ned på URNene. Og de fores inn i avsnittsevaluatoren

# URNs of interest

Variabelen `urns_of_interest` ble definert over, men plassen som fylles kan være hvilken som helst bok. Så det er mulig å hoppe over alt det som gjøres i tidligere celler, og gå rett på analysen av bøker som allerede er av interesse, som Collett eller Skram, eller.

In [32]:
para_of_interest = interesting_paragraphs(urns_of_interest[:10], Kvinde_dannelse, 3)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
para_of_interest

Her henter vi ut de paragrafene som har nok treff.

In [158]:
paras = get_para(para_of_interest)

Noen linjer for å lage ord fra temadefinisjon

In [159]:
wordhits = []
for ws in Kvinnelig_sandhed.values():
    wordhits += ws


In [160]:
def para2html(paras, wordhits):
    from IPython.display import HTML
    html = ""
    for urn in paras:
        metadata = nb.metadata(urn)[0]
        item = "<h4><a href='{lnk}' target='_blank'>{meta}</a></h4>".format(
            lnk = link(urn),
            meta = ', '.join([str(x) for x in metadata]))
        content = ""
        for p in paras[urn]:
            q = paras[urn][p]
            for word in wordhits:
                q = q.replace(word, "<mark>"+word+"</mark>")
            content += "<p>{paragraph}</p>".format(paragraph = q)
        html += item + content
    return HTML("<body>" + html + "</body>")

In [161]:
para2html(paras, wordhits)